In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
import random
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.label = df.label.astype("object")
df.station_id = df.station_id.astype("object")
df.line_id = df.line_id.astype("object")
df.ticket_id = df.ticket_id.astype("object")
df.id = df.id.astype("object")

In [4]:
del df['id']

In [5]:
df.entrance_id = df.entrance_id.astype("object")

In [6]:
df['year'] = pd.DatetimeIndex(df['pass_dttm']).year
df['month'] = pd.DatetimeIndex(df['pass_dttm']).month
df['day'] = pd.DatetimeIndex(df['pass_dttm']).day
df['weekday'] = pd.DatetimeIndex(df['pass_dttm']).weekday
df['hour'] = pd.DatetimeIndex(df['pass_dttm']).hour
df['minute'] = pd.DatetimeIndex(df['pass_dttm']).minute

In [7]:
del df['pass_dttm']

In [8]:
df.weekday = df.weekday.astype("object")

In [9]:
df.head()

,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,time_to_under,label,year,month,weekday,hour,minute
0,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,216.316667,8001,2022,9,0,5,0
1,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,648.183333,9011,2022,9,0,5,0
2,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,865.333333,7022,2022,9,0,5,0
3,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,1048.233333,2022,2022,9,0,5,1
4,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,965.600000,2017,2022,9,0,5,2


In [10]:
del df['station_nm']
del df['line_nm']
del df['entrance_nm']
del df['ticket_id']
del df['minute']

In [11]:
features = ["year", "month", "hour"]
target = ["time_to_under", "label"]
category = []
for i in df.columns:
    if i not in features and i not in target:
        category.append(i)

In [12]:
'''
for i in category:
    print(i)
    df[i] = pd.Categorical(df[i])
    df[i].astype('category').cat.codes
    df[i] = df[i].cat.codes
'''

"\nfor i in category:\n    print(i)\n    df[i] = pd.Categorical(df[i])\n    df[i].astype('category').cat.codes\n    df[i] = df[i].cat.codes\n"

In [13]:
category

['ticket_type_nm', 'entrance_id', 'station_id', 'line_id', 'weekday']

In [14]:
from category_encoders.binary import BinaryEncoder
d = pd.DataFrame()
ct = []
binens = []
for i in category:
    print(i)
    bn = BinaryEncoder()
    bn.fit(df[i].values)
    binens.append(bn)
    k = bn.transform(df[i].values)
    l  = []
    for j in k.columns:
        l.append(str(i)+str(j))
    ct.extend(l)
    k.columns = l
    d = pd.concat([d, k], axis=1)

ticket_type_nm
entrance_id
station_id
line_id
weekday


In [15]:
feat = features.copy()
feat.extend(target)

In [16]:
df = df[feat]

In [17]:
df = pd.concat([df, d], axis=1)

In [18]:
np.random.seed(77)
val_ds = pd.DataFrame()
train_ds = pd.DataFrame()
k = 0
l = len(df.label.unique())
for label in df.label.unique():
    k+=1
    if k%int(l/25)==0: print(k)
    #n = round(0.2*len(df.loc[df['label']==label]))
    n = 5
    ids = random.sample(list(df.loc[df.label == label].index), n)
    val_ds = pd.concat([val_ds, df.loc[ids]])
    train_ids = list(set(df.loc[df.label == label].index) - set(ids))
    train_ds = pd.concat([train_ds, df.loc[train_ids]])

11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
264
275


In [71]:
val_ds.shape[0]/train_ds.shape[0]

0.0012664721683563669

In [72]:
train_ds['label'] = train_ds['label'].astype('int64')

In [73]:
train = train_ds.copy()
y = train['time_to_under']
del train['time_to_under']
del train['label']
X = train

In [74]:
X.head(2)

,year,month,hour,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,ticket_type_nm0_5,entrance_id0_0,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,5,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,1
851969,2022,9,6,0,1,0,0,0,1,0,...,1,1,0,0,0,1,0,1,1,0


In [75]:
model = CatBoostRegressor(iterations=500,
                              cat_features = ct,
                             task_type = 'GPU')

In [76]:
model.fit(X,y)

Learning rate set to 0.139319
0:	learn: 217.9962041	total: 17.5ms	remaining: 8.74s
1:	learn: 206.2653354	total: 35.8ms	remaining: 8.92s
2:	learn: 197.1458152	total: 54.2ms	remaining: 8.98s
3:	learn: 189.7116172	total: 72.5ms	remaining: 9s
4:	learn: 184.1118904	total: 90ms	remaining: 8.91s
5:	learn: 179.8643044	total: 108ms	remaining: 8.91s
6:	learn: 176.5538048	total: 125ms	remaining: 8.83s
7:	learn: 173.6494179	total: 143ms	remaining: 8.8s
8:	learn: 171.5267553	total: 160ms	remaining: 8.71s
9:	learn: 169.8631232	total: 177ms	remaining: 8.68s
10:	learn: 168.5911924	total: 195ms	remaining: 8.65s
11:	learn: 167.6020328	total: 211ms	remaining: 8.58s
12:	learn: 166.8198469	total: 228ms	remaining: 8.55s
13:	learn: 165.5547168	total: 246ms	remaining: 8.54s
14:	learn: 164.8463102	total: 264ms	remaining: 8.53s
15:	learn: 164.4494717	total: 285ms	remaining: 8.63s
16:	learn: 164.1410124	total: 304ms	remaining: 8.64s
17:	learn: 163.4620055	total: 323ms	remaining: 8.66s
18:	learn: 163.2339349	tota

159:	learn: 159.3024675	total: 3.08s	remaining: 6.54s
160:	learn: 159.2967216	total: 3.1s	remaining: 6.52s
161:	learn: 159.2942910	total: 3.11s	remaining: 6.5s
162:	learn: 159.2897070	total: 3.13s	remaining: 6.48s
163:	learn: 159.2858250	total: 3.16s	remaining: 6.47s
164:	learn: 159.2788925	total: 3.18s	remaining: 6.45s
165:	learn: 159.2746739	total: 3.2s	remaining: 6.44s
166:	learn: 159.2652687	total: 3.22s	remaining: 6.42s
167:	learn: 159.2593621	total: 3.24s	remaining: 6.41s
168:	learn: 159.2549600	total: 3.26s	remaining: 6.39s
169:	learn: 159.2516731	total: 3.28s	remaining: 6.37s
170:	learn: 159.2472708	total: 3.31s	remaining: 6.36s
171:	learn: 159.2434822	total: 3.33s	remaining: 6.34s
172:	learn: 159.2389617	total: 3.35s	remaining: 6.33s
173:	learn: 159.2334672	total: 3.37s	remaining: 6.31s
174:	learn: 159.2291175	total: 3.39s	remaining: 6.3s
175:	learn: 159.2254996	total: 3.41s	remaining: 6.28s
176:	learn: 159.2215570	total: 3.44s	remaining: 6.27s
177:	learn: 159.2172778	total: 3

319:	learn: 158.7802333	total: 6.34s	remaining: 3.57s
320:	learn: 158.7786648	total: 6.37s	remaining: 3.55s
321:	learn: 158.7776172	total: 6.39s	remaining: 3.53s
322:	learn: 158.7761316	total: 6.41s	remaining: 3.51s
323:	learn: 158.7732077	total: 6.42s	remaining: 3.49s
324:	learn: 158.7714143	total: 6.44s	remaining: 3.47s
325:	learn: 158.7663594	total: 6.46s	remaining: 3.45s
326:	learn: 158.7631275	total: 6.48s	remaining: 3.43s
327:	learn: 158.7604994	total: 6.5s	remaining: 3.41s
328:	learn: 158.7568767	total: 6.52s	remaining: 3.39s
329:	learn: 158.7546391	total: 6.54s	remaining: 3.37s
330:	learn: 158.7521351	total: 6.56s	remaining: 3.35s
331:	learn: 158.7491634	total: 6.59s	remaining: 3.33s
332:	learn: 158.7479972	total: 6.61s	remaining: 3.31s
333:	learn: 158.7458542	total: 6.63s	remaining: 3.29s
334:	learn: 158.7437408	total: 6.65s	remaining: 3.27s
335:	learn: 158.7404019	total: 6.67s	remaining: 3.25s
336:	learn: 158.7377201	total: 6.69s	remaining: 3.23s
337:	learn: 158.7359085	total

478:	learn: 158.4562373	total: 9.59s	remaining: 420ms
479:	learn: 158.4547250	total: 9.61s	remaining: 400ms
480:	learn: 158.4517714	total: 9.63s	remaining: 380ms
481:	learn: 158.4498617	total: 9.65s	remaining: 360ms
482:	learn: 158.4479341	total: 9.67s	remaining: 340ms
483:	learn: 158.4467123	total: 9.69s	remaining: 320ms
484:	learn: 158.4451406	total: 9.71s	remaining: 300ms
485:	learn: 158.4433849	total: 9.73s	remaining: 280ms
486:	learn: 158.4403303	total: 9.75s	remaining: 260ms
487:	learn: 158.4383789	total: 9.77s	remaining: 240ms
488:	learn: 158.4367003	total: 9.79s	remaining: 220ms
489:	learn: 158.4349209	total: 9.81s	remaining: 200ms
490:	learn: 158.4327677	total: 9.83s	remaining: 180ms
491:	learn: 158.4310832	total: 9.86s	remaining: 160ms
492:	learn: 158.4287164	total: 9.88s	remaining: 140ms
493:	learn: 158.4270852	total: 9.9s	remaining: 120ms
494:	learn: 158.4250921	total: 9.92s	remaining: 100ms
495:	learn: 158.4236625	total: 9.94s	remaining: 80.1ms
496:	learn: 158.4221083	tota

In [77]:
# посмотрим на важность признаков
imp = pd.DataFrame({
    'feature_name':X.columns,
    'feature_score': model.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

imp.to_numpy()

array([['hour', 45.87206307089954],
       ['ticket_type_nm0_5', 13.02165525050484],
       ['ticket_type_nm0_3', 9.225548962955553],
       ['ticket_type_nm0_4', 8.315204299215361],
       ['ticket_type_nm0_1', 6.701902103546824],
       ['weekday0_0', 2.9028726815406287],
       ['ticket_type_nm0_2', 2.50075472554405],
       ['weekday0_1', 2.3472237682840515],
       ['line_id0_2', 0.656422808702844],
       ['line_id0_3', 0.6410827368194003],
       ['weekday0_2', 0.6114220471158482],
       ['ticket_type_nm0_0', 0.5822043358749437],
       ['line_id0_4', 0.5357582079346238],
       ['line_id0_1', 0.5312059182424698],
       ['station_id0_5', 0.5099918858454534],
       ['station_id0_7', 0.4202041959588598],
       ['entrance_id0_0', 0.4116344424212518],
       ['station_id0_3', 0.3800598613739865],
       ['station_id0_4', 0.36186467327963767],
       ['station_id0_6', 0.33485100686272634],
       ['entrance_id0_3', 0.32801904555816297],
       ['station_id0_8', 0.3239757191849535

In [78]:
from sklearn.metrics import r2_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
del X_test['time_to_under']
pred = model.predict(X_test)
r2 = r2_score(val_ds.time_to_under.to_list(), pred)
print(model,'\n',r2_score(val_ds.time_to_under.to_list(), pred))

 0.5221324467946304


In [39]:
train = train_ds.copy()
y = train['label']
#del train['time_to_under']
del train['label']
X = train

In [40]:
X.head()

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,216.316667,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
851969,2022,9,17,6,697.716667,0,1,0,0,0,...,1,1,0,0,0,1,0,1,1,0
589832,2022,9,15,8,602.350000,0,1,0,0,0,...,0,0,0,1,0,0,1,1,0,0
655383,2022,9,15,13,343.016667,0,1,0,1,1,...,1,0,0,1,0,1,1,1,0,0
688151,2022,9,16,6,771.666667,0,0,1,1,1,...,0,1,0,0,0,1,0,1,0,1


In [41]:
clf = RandomForestClassifier(random_state=77, n_estimators=10)

In [42]:
clf.fit(X.to_numpy(), y.to_numpy())

RandomForestClassifier(n_estimators=10, random_state=77)

In [43]:
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 4.26665178e-02, 1.69171750e-01,
       5.80141126e-01, 3.83361555e-03, 1.84392784e-02, 2.36228792e-02,
       1.76868975e-02, 2.51226730e-02, 3.23810429e-02, 9.68850457e-04,
       1.18408734e-03, 1.66589848e-03, 1.57521509e-03, 1.35511468e-03,
       1.99457473e-03, 1.93927742e-03, 1.97566587e-03, 1.93770746e-03,
       1.20183229e-04, 9.74938482e-04, 1.22024300e-03, 1.19916535e-03,
       1.28990544e-03, 1.25445412e-03, 1.39474367e-03, 1.42521398e-03,
       1.42699805e-03, 6.25068468e-06, 1.57538091e-03, 1.14617583e-03,
       1.18121555e-03, 1.26890068e-03, 1.60955492e-02, 1.85051292e-02,
       2.22533805e-02])

In [44]:
from sklearn.metrics import recall_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
X_test['time_to_under'] = pred
pred1 = clf.predict(X_test)
recall = recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0)
print(clf,'\n',recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0))

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(n_estimators=10, random_state=77) 
 0.04057971014492753


In [45]:
res = 0.5*r2+0.5*recall
res

0.2857499022917061

In [44]:
#Обучим на всей выборке

In [19]:
train = df.copy()
y = train['time_to_under']
del train['time_to_under']
del train['label']
X = train

In [20]:
X.head(1)

,year,month,hour,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,ticket_type_nm0_5,entrance_id0_0,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,5,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,1


In [21]:
model = CatBoostRegressor(iterations=500,
                              cat_features = ct,
                             task_type = 'GPU')

In [22]:
 model.fit(X,y)

Learning rate set to 0.139328
0:	learn: 217.9932855	total: 17.6ms	remaining: 8.77s
1:	learn: 206.2625493	total: 34.4ms	remaining: 8.57s
2:	learn: 197.1431714	total: 51.2ms	remaining: 8.48s
3:	learn: 189.7091267	total: 68.4ms	remaining: 8.48s
4:	learn: 184.1096632	total: 85ms	remaining: 8.42s
5:	learn: 179.8625556	total: 101ms	remaining: 8.35s
6:	learn: 176.5523691	total: 118ms	remaining: 8.29s
7:	learn: 173.6485601	total: 135ms	remaining: 8.31s
8:	learn: 171.5262957	total: 152ms	remaining: 8.27s
9:	learn: 169.8626907	total: 169ms	remaining: 8.27s
10:	learn: 168.5910632	total: 186ms	remaining: 8.27s
11:	learn: 167.5943319	total: 203ms	remaining: 8.24s
12:	learn: 166.8121170	total: 220ms	remaining: 8.25s
13:	learn: 165.5495434	total: 238ms	remaining: 8.26s
14:	learn: 164.8625145	total: 255ms	remaining: 8.25s
15:	learn: 164.4474869	total: 272ms	remaining: 8.24s
16:	learn: 163.8386512	total: 289ms	remaining: 8.22s
17:	learn: 163.3856927	total: 306ms	remaining: 8.19s
18:	learn: 163.1546691	

156:	learn: 159.3333201	total: 3.28s	remaining: 7.16s
157:	learn: 159.3310817	total: 3.3s	remaining: 7.14s
158:	learn: 159.3265988	total: 3.32s	remaining: 7.12s
159:	learn: 159.3236710	total: 3.34s	remaining: 7.1s
160:	learn: 159.3185576	total: 3.36s	remaining: 7.08s
161:	learn: 159.3163366	total: 3.38s	remaining: 7.05s
162:	learn: 159.3130846	total: 3.4s	remaining: 7.03s
163:	learn: 159.3090431	total: 3.42s	remaining: 7.01s
164:	learn: 159.3042709	total: 3.44s	remaining: 6.99s
165:	learn: 159.2957984	total: 3.46s	remaining: 6.96s
166:	learn: 159.2878617	total: 3.48s	remaining: 6.94s
167:	learn: 159.2802840	total: 3.5s	remaining: 6.92s
168:	learn: 159.2687283	total: 3.52s	remaining: 6.9s
169:	learn: 159.2637191	total: 3.54s	remaining: 6.88s
170:	learn: 159.2603128	total: 3.56s	remaining: 6.86s
171:	learn: 159.2581735	total: 3.59s	remaining: 6.84s
172:	learn: 159.2533173	total: 3.61s	remaining: 6.82s
173:	learn: 159.2479541	total: 3.63s	remaining: 6.8s
174:	learn: 159.2441231	total: 3.6

318:	learn: 158.7964349	total: 6.64s	remaining: 3.77s
319:	learn: 158.7945199	total: 6.66s	remaining: 3.75s
320:	learn: 158.7893776	total: 6.68s	remaining: 3.72s
321:	learn: 158.7861148	total: 6.7s	remaining: 3.7s
322:	learn: 158.7828578	total: 6.72s	remaining: 3.68s
323:	learn: 158.7804638	total: 6.74s	remaining: 3.66s
324:	learn: 158.7770354	total: 6.76s	remaining: 3.64s
325:	learn: 158.7747536	total: 6.78s	remaining: 3.62s
326:	learn: 158.7718038	total: 6.8s	remaining: 3.6s
327:	learn: 158.7698412	total: 6.82s	remaining: 3.58s
328:	learn: 158.7665189	total: 6.84s	remaining: 3.55s
329:	learn: 158.7640774	total: 6.86s	remaining: 3.53s
330:	learn: 158.7606367	total: 6.88s	remaining: 3.51s
331:	learn: 158.7595726	total: 6.9s	remaining: 3.49s
332:	learn: 158.7571723	total: 6.92s	remaining: 3.47s
333:	learn: 158.7554874	total: 6.94s	remaining: 3.45s
334:	learn: 158.7539680	total: 6.96s	remaining: 3.42s
335:	learn: 158.7507695	total: 6.97s	remaining: 3.4s
336:	learn: 158.7493801	total: 7s	

471:	learn: 158.4769866	total: 9.79s	remaining: 581ms
472:	learn: 158.4760331	total: 9.8s	remaining: 560ms
473:	learn: 158.4737173	total: 9.82s	remaining: 539ms
474:	learn: 158.4719406	total: 9.85s	remaining: 518ms
475:	learn: 158.4704718	total: 9.87s	remaining: 498ms
476:	learn: 158.4692872	total: 9.89s	remaining: 477ms
477:	learn: 158.4677828	total: 9.91s	remaining: 456ms
478:	learn: 158.4660415	total: 9.93s	remaining: 435ms
479:	learn: 158.4650879	total: 9.95s	remaining: 415ms
480:	learn: 158.4635421	total: 9.97s	remaining: 394ms
481:	learn: 158.4620139	total: 9.99s	remaining: 373ms
482:	learn: 158.4607049	total: 10s	remaining: 353ms
483:	learn: 158.4586082	total: 10s	remaining: 332ms
484:	learn: 158.4569852	total: 10.1s	remaining: 311ms
485:	learn: 158.4553504	total: 10.1s	remaining: 290ms
486:	learn: 158.4527679	total: 10.1s	remaining: 270ms
487:	learn: 158.4502504	total: 10.1s	remaining: 249ms
488:	learn: 158.4485978	total: 10.1s	remaining: 228ms
489:	learn: 158.4473005	total: 10

In [23]:
# посмотрим на важность признаков
imp = pd.DataFrame({
    'feature_name':X.columns,
    'feature_score': model.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

imp.to_numpy()

array([['hour', 45.67665402802449],
       ['ticket_type_nm0_5', 13.316729660027303],
       ['ticket_type_nm0_3', 9.310348163787236],
       ['ticket_type_nm0_4', 7.9196172231998005],
       ['ticket_type_nm0_1', 7.332915005845077],
       ['weekday0_0', 2.864703743357808],
       ['weekday0_1', 2.305426823450294],
       ['ticket_type_nm0_2', 2.293529516842965],
       ['weekday0_2', 0.5995323914167534],
       ['ticket_type_nm0_0', 0.5971805735386829],
       ['line_id0_2', 0.5842320282162867],
       ['line_id0_3', 0.5517003428321401],
       ['line_id0_4', 0.5180451797537223],
       ['line_id0_1', 0.49110316177297625],
       ['station_id0_7', 0.4431540422537238],
       ['entrance_id0_0', 0.4250604019089043],
       ['station_id0_5', 0.4106286472460166],
       ['station_id0_3', 0.40914814132825583],
       ['station_id0_6', 0.37047210302347466],
       ['station_id0_8', 0.36221855216922494],
       ['entrance_id0_3', 0.32124833280991694],
       ['station_id0_2', 0.321222626736

In [25]:
from sklearn.metrics import r2_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
del X_test['time_to_under']
pred = model.predict(X_test)
print(pred)
#X = X_test['postmark'].copy()
#X.loc[X==0.0] = 0
#X.loc[X!=0] = 1
#pred = pr(X.to_list())
r2 = r2_score(val_ds.time_to_under.to_list(), pred)
print(model,'\n',r2_score(val_ds.time_to_under.to_list(), pred))

[567.79565 598.4594  680.2964  ... 401.8845  135.56525 496.40994]
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...) 
 0.5365789940403094


In [26]:
train = df.copy()
train['label'] = train['label'].astype('int64')
y = train['label']
#del train['time_to_under']
del train['label']
X = train

In [27]:
clf = RandomForestClassifier(random_state=77, n_estimators=10)

In [28]:
X.head(1)

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,216.316667,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [29]:
clf.fit(X.to_numpy(), y.to_numpy())

RandomForestClassifier(n_estimators=10, random_state=77)

In [30]:
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 4.29265434e-02, 1.66837565e-01,
       5.78209689e-01, 3.87340243e-03, 1.86534448e-02, 2.41389378e-02,
       1.82975095e-02, 2.57317110e-02, 3.28948622e-02, 9.65797439e-04,
       1.24248986e-03, 1.83018262e-03, 1.58902122e-03, 1.46876059e-03,
       1.94834999e-03, 1.85295223e-03, 2.05503097e-03, 2.02938596e-03,
       1.22413724e-04, 1.07002821e-03, 1.29872990e-03, 1.43392775e-03,
       1.30504751e-03, 1.35262635e-03, 1.44473465e-03, 1.68913228e-03,
       1.47824476e-03, 9.37515494e-06, 1.56790827e-03, 9.98829925e-04,
       1.13725421e-03, 1.26879325e-03, 1.60472120e-02, 1.87723692e-02,
       2.24577376e-02])

In [31]:
from sklearn.metrics import recall_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
#del X_test['ticket_id']
X_test['time_to_under'] = pred
pred1 = clf.predict(X_test)
print(pred1)
#X = X_test['postmark'].copy()
#X.loc[X==0.0] = 0
#X.loc[X!=0] = 1
#pred = pr(X.to_list())
recall = recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0)
print(clf,'\n',recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0))

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[ 1022  6015  8006 ...  6006 13006 13001]
RandomForestClassifier(n_estimators=10, random_state=77) 
 0.24420289855072466


In [32]:
res = 0.5*r2+0.5*recall
res

0.390390946295517

In [32]:
#сделаем предсказание на тестовом датасете

In [33]:
test_df = pd.read_csv("test.csv")

In [34]:
test_df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm
0,1091022,425A13E16226CA01032043111488E0F5,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:03
1,1091023,F94EDC107A4E054786882F9453C1B93F,Пропуск FacePay,184,Молодёжная (Западный),3010,Молодёжная,3,Арбатско-Покровская,2022-09-12 05:00:45
2,1091024,8F13BFBA98A7DD34B11BD2DDCC5CA68E,СК студента 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:52
3,1091025,7B98691636EC418ABF6FEDDE4D7C68FC,КОШЕЛЕК,107,Сокол ( Южн./Вост. ),2018,Сокол,2,Замоскворецкая,2022-09-12 05:00:58
4,1091026,679C55C28CB36FB546FD104502C0B3DB,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:39


In [35]:
test_df.shape

(561890, 10)

In [36]:
test_df.station_id = test_df.station_id.astype("object")
test_df.line_id = test_df.line_id.astype("object")
test_df.ticket_id = test_df.ticket_id.astype("object")
test_df.entrance_id = test_df.entrance_id.astype("object")

In [37]:
test_df['year'] = pd.DatetimeIndex(test_df['pass_dttm']).year
test_df['month'] = pd.DatetimeIndex(test_df['pass_dttm']).month
test_df['day'] = pd.DatetimeIndex(test_df['pass_dttm']).day
test_df['weekday'] = pd.DatetimeIndex(test_df['pass_dttm']).weekday
test_df['hour'] = pd.DatetimeIndex(test_df['pass_dttm']).hour
test_df['minute'] = pd.DatetimeIndex(test_df['pass_dttm']).minute

In [38]:
del test_df['pass_dttm']

In [39]:
test_df.weekday = test_df.weekday.astype("object")

In [40]:
del test_df['station_nm']
del test_df['line_nm']
del test_df['entrance_nm']
del test_df['ticket_id']
del test_df['minute']

In [41]:
from category_encoders.binary import BinaryEncoder
d = pd.DataFrame()
ct = []
#binens = []

for i in range(0, len(category)):
    print(category[i])
    if i != "name_mfi":
        k = binens[i].transform(test_df[category[i]].values)
        l  = []
        for j in k.columns:
            l.append(str(category[i])+str(j))
        ct.extend(l)
        k.columns = l
        d = pd.concat([d, k], axis=1)

ticket_type_nm
entrance_id
station_id
line_id
weekday


In [42]:
test_features = features.copy()
test_features.append("id")

In [43]:
test_df = test_df[test_features]

In [44]:
test_df = pd.concat([test_df, d], axis=1)

In [45]:
test_df.shape

(561890, 37)

In [46]:
X = test_df.copy()

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091021 entries, 0 to 1091020
Data columns (total 38 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   year               1091021 non-null  int64  
 1   month              1091021 non-null  int64  
 2   day                1091021 non-null  int64  
 3   hour               1091021 non-null  int64  
 4   time_to_under      1091021 non-null  float64
 5   label              1091021 non-null  object 
 6   ticket_type_nm0_0  1091021 non-null  int64  
 7   ticket_type_nm0_1  1091021 non-null  int64  
 8   ticket_type_nm0_2  1091021 non-null  int64  
 9   ticket_type_nm0_3  1091021 non-null  int64  
 10  ticket_type_nm0_4  1091021 non-null  int64  
 11  ticket_type_nm0_5  1091021 non-null  int64  
 12  entrance_id0_0     1091021 non-null  int64  
 13  entrance_id0_1     1091021 non-null  int64  
 14  entrance_id0_2     1091021 non-null  int64  
 15  entrance_id0_3     1091021 non-n

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561890 entries, 0 to 561889
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   year               561890 non-null  int64
 1   month              561890 non-null  int64
 2   day                561890 non-null  int64
 3   hour               561890 non-null  int64
 4   id                 561890 non-null  int64
 5   ticket_type_nm0_0  561890 non-null  int64
 6   ticket_type_nm0_1  561890 non-null  int64
 7   ticket_type_nm0_2  561890 non-null  int64
 8   ticket_type_nm0_3  561890 non-null  int64
 9   ticket_type_nm0_4  561890 non-null  int64
 10  ticket_type_nm0_5  561890 non-null  int64
 11  entrance_id0_0     561890 non-null  int64
 12  entrance_id0_1     561890 non-null  int64
 13  entrance_id0_2     561890 non-null  int64
 14  entrance_id0_3     561890 non-null  int64
 15  entrance_id0_4     561890 non-null  int64
 16  entrance_id0_5     561890 non-null  in

In [49]:
del X['id']

In [50]:
pred = model.predict(X)
print(pred)

[693.7107  487.9439  639.9513  ... 527.30524  70.50696  82.10585]


In [51]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1380 entries, 995678 to 414385
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               1380 non-null   int64  
 1   month              1380 non-null   int64  
 2   day                1380 non-null   int64  
 3   hour               1380 non-null   int64  
 4   time_to_under      1380 non-null   float32
 5   ticket_type_nm0_0  1380 non-null   int64  
 6   ticket_type_nm0_1  1380 non-null   int64  
 7   ticket_type_nm0_2  1380 non-null   int64  
 8   ticket_type_nm0_3  1380 non-null   int64  
 9   ticket_type_nm0_4  1380 non-null   int64  
 10  ticket_type_nm0_5  1380 non-null   int64  
 11  entrance_id0_0     1380 non-null   int64  
 12  entrance_id0_1     1380 non-null   int64  
 13  entrance_id0_2     1380 non-null   int64  
 14  entrance_id0_3     1380 non-null   int64  
 15  entrance_id0_4     1380 non-null   int64  
 16  entrance_id0_5   

In [52]:
X = X[features]
X['time_to_under'] = pred
X = pd.concat([X, test_df[ct]], axis=1)

In [53]:
X.head()

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,693.710693,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
1,2022,9,12,5,487.943909,0,0,0,0,0,...,1,1,0,0,1,1,0,0,0,1
2,2022,9,12,5,639.951294,0,0,0,1,1,...,1,0,0,0,0,1,0,0,0,1
3,2022,9,12,5,705.138489,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
4,2022,9,12,5,693.710693,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [54]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561890 entries, 0 to 561889
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   year               561890 non-null  int64  
 1   month              561890 non-null  int64  
 2   day                561890 non-null  int64  
 3   hour               561890 non-null  int64  
 4   time_to_under      561890 non-null  float32
 5   ticket_type_nm0_0  561890 non-null  int64  
 6   ticket_type_nm0_1  561890 non-null  int64  
 7   ticket_type_nm0_2  561890 non-null  int64  
 8   ticket_type_nm0_3  561890 non-null  int64  
 9   ticket_type_nm0_4  561890 non-null  int64  
 10  ticket_type_nm0_5  561890 non-null  int64  
 11  entrance_id0_0     561890 non-null  int64  
 12  entrance_id0_1     561890 non-null  int64  
 13  entrance_id0_2     561890 non-null  int64  
 14  entrance_id0_3     561890 non-null  int64  
 15  entrance_id0_4     561890 non-null  int64  
 16  en

In [55]:
pred1 = clf.predict(X)

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [56]:
pred1

array([9010, 3016, 2015, ..., 1020, 6015, 2011], dtype=int64)

In [57]:
sample = pd.read_csv("sample_solution.csv")

In [58]:
sample.head()

,id,time_to_under,label
0,1091022,0,0
1,1091023,0,0
2,1091024,0,0
3,1091025,0,0
4,1091026,0,0


In [59]:
result = pd.DataFrame({"id": test_df.id.to_list(), "time_to_under": pred, "label": pred1})

In [60]:
result.head()

,id,time_to_under,label
0,1091022,693.710693,9010
1,1091023,487.943909,3016
2,1091024,639.951294,2015
3,1091025,705.138489,2005
4,1091026,693.710693,9010


In [61]:
result.to_csv("submission4.csv", index=False)